In [1]:
import tensorflow as tf
tf.random.set_seed(36)


#https://towardsdatascience.com/understanding-semantic-segmentation-with-unet-6be4f42d4b47
import os
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, MaxPooling2D, GlobalMaxPool2D, Conv2D, Conv2DTranspose,concatenate, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.utils import plot_model

## Change the output (decision) to 3-class classifier with softmax activation and sparse categorical cross-entropy loss and reshape the output to vectorized pixels

In [2]:
#https://towardsdatascience.com/understanding-semantic-segmentation-with-unet-6be4f42d4b47
num_class = 3
input_size = (256,256,1)

def conv2d_block(input_tensor, n_filters, kernel_size, batchnorm, activation):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation(activation)(x)
    
    return x
  
def get_unet(input_img, n_filters, dropout, batchnorm, kernel_size, activation, depth):
    # Contracting Path
    c=[None]*(depth+3)
    #form 0(=init) to depth(=last decode), also bottom added as depth[depth+1]
    p=[input_img]*(depth+2)
    #Encode
    for current_depth in range(depth):
        current_depth+=1
        c[current_depth]=conv2d_block(p[current_depth-1], n_filters * pow(2,current_depth-1), kernel_size, batchnorm, activation)
        p[current_depth]=MaxPooling2D((2,2))(c[current_depth])
        p[current_depth]=Dropout(dropout)(p[current_depth])
    #Bottom
    c[depth+1]=conv2d_block(p[depth],n_filters* pow(2,depth),kernel_size,batchnorm, activation)
        
    #Decode
    u=[None]*(depth+2)
    d=[c[depth+1]]*(depth+2)
    for current_depth in reversed(range(depth)):
        u[current_depth] = Conv2DTranspose(n_filters * pow(2,current_depth), (kernel_size, kernel_size), strides = (2, 2), padding = 'same')(d[current_depth+1])
        u[current_depth] = concatenate([u[current_depth], c[current_depth+1]])
        u[current_depth] = Dropout(dropout)(u[current_depth])
        d[current_depth] = conv2d_block(u[current_depth], n_filters * pow(2,current_depth), kernel_size, batchnorm, activation)

    #decision
    outputs = Conv2D(num_class, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(d[0])
    outputs = Conv2D(num_class, 1, activation = 'softmax')(outputs)
    outputs = Reshape([input_size[0]*input_size[1],num_class])(outputs)
    
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [4]:
im_height = 256
im_width  = 256

input_img = Input((im_height, im_width, 1), name = 'img')

# Hyperparameters
filters = [16,32]
#filters = [32]
dropouts = [25]
window_sizes = [3]
activations = ['relu']
depths = [3,5]
#depthts= [7]
for föuter in filters:
    for dropout in dropouts:
        for window_size in window_sizes:
            for activation in activations:
                for depth in depths:
                    string = str(föuter)+'F'+str(dropout)+'Dr'+str(window_size)+'Ws'+activation[:2]+'A'+str(depth)+'D'

                    # Get the UNET
                    model = get_unet(input_img, föuter, dropout/100, True, window_size, activation, depth) 

                    # Compile the UNET
                    model.compile(optimizer = Adam(), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])
                    plot_model(model, to_file='diagrams/'+string+'.png')
                    path='models/U_net_' + string
                    if not os.path.exists(path):
                        os.makedirs(path)
                    model.save(path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/U_net_16F25Dr3WsreA3D\assets
INFO:tensorflow:Assets written to: models/U_net_16F25Dr3WsreA5D\assets
INFO:tensorflow:Assets written to: models/U_net_32F25Dr3WsreA3D\assets
INFO:tensorflow:Assets written to: models/U_net_32F25Dr3WsreA5D\assets
